In [19]:
from typing import Optional, Tuple, List
from dateutil import parser
import numpy as np
import pandas as pd


def to_pri_returns(prices: pd.DataFrame) -> pd.DataFrame:
    """_summary_

    Args:
        prices (pd.DataFrame): _description_

    Returns:
        pd.DataFrame: _description_
    """

    def to_pri_return(price: pd.Series) -> float:
        return price.dropna().pct_change().fillna(0)

    return prices.apply(to_pri_return)


def to_log_returns(prices: pd.DataFrame) -> pd.DataFrame:
    """_summary_

    Args:
        prices (pd.DataFrame): _description_

    Returns:
        pd.DataFrame: _description_
    """
    return to_pri_returns(prices=prices).apply(np.log1p)


def to_num_years(prices: pd.DataFrame) -> float:
    """_summary_

    Args:
        prices (pd.DataFrame): _description_

    Returns:
        float: _description_
    """

    def to_num_year(price) -> float:
        p = price.dropna()
        start = parser.parse(str(p.index[0]))
        end = parser.parse(str(p.index[-1]))
        return (end - start).days / 365.0

    return prices.apply(to_num_year, axis=0)


def to_num_bars(prices: pd.DataFrame) -> float:
    """_summary_

    Args:
        prices (pd.DataFrame): _description_

    Returns:
        float: _description_
    """

    def to_num_bar(price) -> float:
        return len(price.dropna())

    return prices.apply(to_num_bar, axis=0)


def to_ann_factors(prices: pd.DataFrame) -> float:
    """_summary_

    Args:
        prices (pd.DataFrame): _description_

    Returns:
        pd.Series: _description_
    """
    return to_num_bars(prices=prices) / to_num_years(prices=prices)


def to_cum_returns(prices: pd.DataFrame) -> pd.Series:
    """_summary_

    Args:
        prices (pd.DataFrame): _description_

    Returns:
        pd.Series: _description_
    """
    return to_pri_returns(prices=prices).add(1).prod() - 1


def to_ann_returns(prices: pd.DataFrame) -> pd.Series:
    """_summary_

    Args:
        prices (pd.DataFrame): _description_

    Returns:
        pd.Series: _description_
    """
    return (
        to_pri_returns(prices=prices).add(1).prod() ** (1 / to_num_years(prices=prices))
        - 1
    )


def to_ann_variances(
    prices: pd.DataFrame, ann_factors: Optional[float] = None
) -> pd.Series:
    """_summary_

    Args:
        prices (pd.DataFrame): _description_

    Returns:
        pd.Series: _description_
    """
    if not ann_factors:
        ann_factors = to_ann_factors(prices=prices)
    return to_pri_returns(prices=prices).var() * to_ann_factors(prices=prices)


def to_ann_volatilites(
    prices: pd.DataFrame, ann_factors: Optional[float] = None
) -> pd.Series:
    """_summary_

    Args:
        prices (pd.DataFrame): _description_

    Returns:
        pd.Series: _description_
    """
    return to_ann_variances(prices=prices, ann_factors=ann_factors).apply(np.sqrt)


def to_ann_semi_variances(
    prices: pd.DataFrame, ann_factors: Optional[float] = None
) -> pd.Series:
    """_summary_

    Args:
        prices (pd.DataFrame): _description_

    Returns:
        pd.Series: _description_
    """
    pri_returns = to_pri_returns(prices=prices)
    positive_pri_returns = pri_returns[pri_returns >= 0]
    if not ann_factors:
        ann_factors = to_ann_factors(prices=prices)
    return positive_pri_returns.var() * ann_factors


def to_ann_semi_volatilities(
    prices: pd.DataFrame, ann_factors: Optional[float] = None
) -> pd.Series:
    """_summary_

    Args:
        prices (pd.DataFrame): _description_
        ann_factors (Optional[float], optional): _description_. Defaults to None.

    Returns:
        pd.Series: _description_
    """
    return to_ann_semi_variances(prices=prices, ann_factors=ann_factors) ** 0.5


def to_drawdown(
    prices: pd.DataFrame,
    window: Optional[int] = None,
    min_periods: Optional[int] = None,
) -> pd.DataFrame:
    """_summary_

    Args:
        prices (pd.DataFrame): _description_
        window (Optional[int], optional): _description_. Defaults to None.

    Returns:
        pd.Series: _description_
    """
    if window:
        return prices / prices.rolling(window=window, min_periods=min_periods).max() - 1
    return prices / prices.expanding(min_periods=min_periods or 1).max() - 1


def to_max_drawdown(
    prices: pd.DataFrame,
    window: Optional[int] = None,
    min_periods: Optional[int] = None,
) -> pd.Series:
    """_summary_

    Args:
        prices (pd.DataFrame): _description_
        window (Optional[int], optional): _description_. Defaults to None.
        min_periods (Optional[int], optional): _description_. Defaults to None.

    Returns:
        pd.Series: _description_
    """
    return to_drawdown(prices=prices, window=window, min_periods=min_periods).min()


def to_sharpe_ratios(
    prices: pd.DataFrame, risk_free: float = 0.0, ann_factors: Optional[float] = None
) -> pd.Series:
    """_summary_

    Args:
        prices (pd.DataFrame): _description_
        risk_free (float, optional): _description_. Defaults to 0..
        ann_factors (Optional[float], optional): _description_. Defaults to None.

    Returns:
        pd.Series: _description_
    """
    excess_returns = to_ann_returns(prices=prices) - risk_free
    return excess_returns / to_ann_volatilites(prices=prices, ann_factors=ann_factors)


def to_sortino_ratios(
    prices: pd.DataFrame, ann_factors: Optional[float] = None
) -> pd.Series:
    """_summary_

    Args:
        prices (pd.DataFrame): _description_
        ann_factors (Optional[float], optional): _description_. Defaults to None.

    Returns:
        pd.Series: _description_
    """
    if not ann_factors:
        ann_factors = to_ann_factors(prices=prices)

    ann_returns = to_ann_returns(prices=prices)
    ann_semi_volatilities = to_ann_semi_volatilities(
        prices=prices, ann_factors=ann_factors
    )

    return ann_returns / ann_semi_volatilities


def to_tail_ratios(prices: pd.DataFrame, alpha: float = 0.05) -> pd.Series:
    """_summary_

    Args:
        prices (pd.DataFrame): _description_
        alpha (float, optional): _description_. Defaults to 0.05.

    Returns:
        pd.Series: _description_
    """

    def to_tail_ratio(pri_return: pd.Series, alpha: float) -> float:

        r = pri_return.dropna()
        return -r.quantile(q=alpha) / r.quantile(q=1 - alpha)

    return to_pri_returns(prices=prices).apply(to_tail_ratio, alpha=alpha)


def to_skewnesses(prices: pd.DataFrame) -> pd.Series:
    """_summary_

    Args:
        prices (pd.DataFrame): _description_

    Returns:
        pd.Series: _description_
    """

    def to_skewness(pri_return: pd.Series) -> float:

        return pri_return.dropna().skew()

    return to_pri_returns(prices=prices).apply(to_skewness)


def to_kurtosises(prices: pd.DataFrame) -> pd.Series:
    """_summary_

    Args:
        prices (pd.DataFrame): _description_

    Returns:
        pd.Series: _description_
    """

    def to_kurtosis(pri_return: pd.Series) -> float:

        return pri_return.dropna().kurt()

    return to_pri_returns(prices=prices).apply(to_kurtosis)


def to_value_at_risks(prices: pd.DataFrame, alpha: float = 0.05) -> pd.Series:
    """_summary_

    Args:
        prices (pd.DataFrame): _description_
        alpha (float, optional): _description_. Defaults to 0.05.

    Returns:
        pd.Series: _description_
    """
    def to_value_at_risk(pri_return: pd.Series, alpha: float) -> float:

        r = pri_return.dropna()
        return r.quantile(q=alpha)

    return to_pri_returns(prices=prices).apply(to_value_at_risk, alpha=alpha)

def to_expected_shortfalls(prices: pd.DataFrame, alpha: float = 0.05) -> pd.Series:
    
    def to_expected_shortfall(pri_return: pd.Series, alpha: float) -> float:

        r = pri_return.dropna()
        var = r.quantile(q=alpha)
        return r[r <= var].mean()

    return to_pri_returns(prices=prices).apply(to_expected_shortfall, alpha=alpha)    



def cov_to_corr(covariance_matrix: pd.DataFrame) -> pd.DataFrame:
    """_summary_

    Args:
        covariance_matrix (pd.DataFrame): _description_

    Returns:
        pd.DataFrame: _description_
    """
    vol = np.sqrt(np.diag(covariance_matrix))
    corr = covariance_matrix / np.outer(vol, vol)
    corr[corr < -1], corr[corr > 1] = -1, 1
    return corr


def recursive_bisection(sorted_tree) -> List[Tuple[List[int], List[int]]]:
    """_summary_

    Args:
        sorted_tree (_type_): _description_

    Returns:
        List[Tuple[List[int], List[int]]]: _description_
    """

    if len(sorted_tree) < 3:
        return

    num = len(sorted_tree)
    bis = int(num / 2)
    left = sorted_tree[0:bis]
    right = sorted_tree[bis:]
    if len(left) > 2 and len(right) > 2:
        return [(left, right), recursive_bisection(left), recursive_bisection(right)]
    return (left, right)


def get_cluster_assets(clusters, node, num_assets) -> List:
    """_summary_

    Args:
        clusters (_type_): _description_
        node (_type_): _description_
        num_assets (_type_): _description_

    Returns:
        List: _description_
    """
    if node < num_assets:
        return [int(node)]
    row = clusters[int(node - num_assets)]
    return get_cluster_assets(clusters, row[0], num_assets) + get_cluster_assets(
        clusters, row[1], num_assets
    )

In [20]:
import yfinance as yf

prices = yf.download("SPY, QQQ, IWV, MTUM, QUAL, VLUE, IVV")["Adj Close"]
to_expected_shortfalls(prices)


[*********************100%***********************]  7 of 7 completed


IVV    -0.029683
IWV    -0.030038
MTUM   -0.030403
QQQ    -0.041682
QUAL   -0.027506
SPY    -0.028441
VLUE   -0.029014
dtype: float64

In [21]:
to_value_at_risks(prices)


IVV    -0.019268
IWV    -0.019255
MTUM   -0.018910
QQQ    -0.028050
QUAL   -0.016566
SPY    -0.018511
VLUE   -0.018602
dtype: float64

IVV    -1.077696
IWV    -1.054462
MTUM   -1.074796
QQQ    -1.077147
QUAL   -1.030019
SPY    -1.087807
VLUE   -1.069519
dtype: float64

In [21]:
optimizer.risk_parity()


IVV     0.146015
IWV     0.144096
MTUM    0.141989
QQQ     0.127689
QUAL    0.146680
SPY     0.147329
VLUE    0.146203
Name: weights, dtype: float64

In [23]:
cov = optimizer.covariance_matrix
cov


,IVV,IWV,MTUM,QQQ,QUAL,SPY,VLUE
IVV,0.031828,0.032260,0.031828,0.035591,0.031482,0.031414,0.031620
IWV,0.032260,0.032964,0.032430,0.036123,0.031936,0.031855,0.032318
MTUM,0.031828,0.032430,0.038901,0.038143,0.031308,0.031420,0.029708
QQQ,0.035591,0.036123,0.038143,0.046259,0.035353,0.035207,0.032576
QUAL,0.031482,0.031936,0.031308,0.035353,0.032207,0.031123,0.031266
SPY,0.031414,0.031855,0.031420,0.035207,0.031123,0.031061,0.031233
VLUE,0.031620,0.032318,0.029708,0.032576,0.031266,0.031233,0.037283


[[0.03182801 0.03239127 0.03518713 0.03837091 0.0320171  0.03144203
  0.03444766]
 [0.03239127 0.03296449 0.03580984 0.03904996 0.03258371 0.03199845
  0.03505727]
 [0.03518713 0.03580984 0.03890077 0.04242057 0.03539618 0.03476041
  0.03808325]
 [0.03837091 0.03904996 0.04242057 0.04625884 0.03859887 0.03790558
  0.04152908]
 [0.0320171  0.03258371 0.03539618 0.03859887 0.03220732 0.03162882
  0.03465231]
 [0.03144203 0.03199845 0.03476041 0.03790558 0.03162882 0.03106072
  0.0340299 ]
 [0.03444766 0.03505727 0.03808325 0.04152908 0.03465231 0.0340299
  0.03728291]]


,IVV,IWV,MTUM,QQQ,QUAL,SPY,VLUE
IVV,1.000000,0.995939,0.904524,0.927553,0.983290,0.999117,0.917913
IWV,0.995939,1.000000,0.905626,0.925056,0.980130,0.995516,0.921854
MTUM,0.904524,0.905626,1.000000,0.899166,0.884501,0.903899,0.780070
QQQ,0.927553,0.925056,0.899166,1.000000,0.915910,0.928798,0.784404
QUAL,0.983290,0.980130,0.884501,0.915910,1.000000,0.984018,0.902273
SPY,0.999117,0.995516,0.903899,0.928798,0.984018,1.000000,0.917825
VLUE,0.917913,0.921854,0.780070,0.784404,0.902273,0.917825,1.000000


In [4]:
def quasi_diagnalization(clusters, num_assets, curr_index):
    """Rearrange the assets to reorder them according to hierarchical tree clustering order"""

    if curr_index < num_assets:
        return [curr_index]

    left = int(clusters[curr_index - num_assets, 0])
    right = int(clusters[curr_index - num_assets, 1])

    return quasi_diagnalization(clusters, num_assets, left) + quasi_diagnalization(
        clusters, num_assets, right
    )


from scipy.cluster.hierarchy import to_tree, linkage, dendrogram
import yfinance as yf

prices = yf.download("SPY, QQQ, IWV, MTUM, QUAL, VLUE, IVV")["Adj Close"].dropna()
expected_returns = prices.pct_change().fillna(0).mean() * 252
covariance_matrix = prices.pct_change().fillna(0).cov() * (252)
corr = cov_to_corr(covariance_matrix.values)
dist = np.sqrt((1 - corr).round(5) / 2)
clusters = linkage(squareform(dist), method="single")
sorted_index = list(to_tree(clusters, rd=False).pre_order())
# clustered_assets = [[self.assets[x] for x in sorted_index]]
# print(sorted_index)
# dendrogram(clusters)

[*********************100%***********************]  7 of 7 completed


In [11]:
from typing import Tuple


print(recursive_bisection(sorted_index))

[([2, 6, 3], [4, 1, 0, 5]), [([2], [6, 3])], [([4, 1], [0, 5])]]


In [ ]:
sorted_index


def divide_chunks(l, n):

    # looping till length l
    for i in range(0, len(l), n):
        yield l[i : i + n]


for i in range(len(sorted_index) - 2, 0, -1):

    print(i)

    print(list(divide_chunks(sorted_index, i)))
    # print(list(divide_chunks(sorted_index, i)))

In [ ]:
def inverse_variance_weights(covariance_matrix: np.ndarray) -> np.ndarray:
    """calculate weights of inverse variance. (tot weights = 100%)

    Args:
        covariance_matrix (np.ndarray): _description_

    Returns:
        np.ndarray: weights of
    """
    inv_var_weights = 1 / np.diag(covariance_matrix)
    inv_var_weights /= inv_var_weights.sum()
    return inv_var_weights


inverse_variance_weights(covariance_matrix.values)

In [ ]:
import pandas as pd

meta = pd.read_excel("database.xlsx", sheet_name="tb_meta")

import yfinance as yf

# for id, m in meta.iterrows():
#     tic = yf.Ticker(m.ticker)
#     ff = tic.isin
#     print(ff)

for id, row in meta.iterrows():
    print(row.ticker)
    if row.__ticker.endswith(".KS"):
        continue

    t = yf.Ticker(row.ticker)

    hist = t.history(period="1y")
    try:
        splits = hist["Stock Splits"]

        test = (splits != 0).any()
        if test:
            print("Has splits")
            print(row.ticker)
    except:
        continue

In [ ]:
import yfinance as yf

t = yf.Ticker("HYMB")
hist = t.history(period="1y")
splits = hist["Stock Splits"]
splits = splits[splits != 0]
splits

In [ ]:
""" base strategy class """

import warnings
from typing import Any, List, Optional, Dict
import numpy as np
import pandas as pd


class Account:
    """strategy account"""

    def __init__(self):
        self.date: List = []
        self.value: List = []
        self.weights: List[Dict] = []
        self.reb_weights: List[Dict] = []
        self.trade_weights: List[Dict] = []


class BaseStrategy:
    """
    BaseStrategy class is an algorithmic trading strategy that sequentially allocates capital among
    group of assets based on pre-defined allocatioin method.

    BaseStrategy shall be the parent class for all investment strategies with period-wise
    rebalancing scheme.

    Using this class requires following pre-defined methods:
    1. rebalance(self, price_asset, date, **kwargs):
        the method shall be in charge of calculating new weights based on prices
        that are avaiable.
    2. monitor (self, ...):
        the method shall be in charge of monitoring the strategy status, and if
        necessary call the rebalance method to re-calculate weights. aka irregular rebalancing.
    """

    def __init__(
        self,
        price_asset: pd.DataFrame,
        frequency: str = "M",
        min_assets: int = 2,
        min_periods: int = 2,
        investment: float = 1000.0,
        commission: int = 0,
        currency: str = "KRW",
        name: str = "strategy",
        account: Optional[Account] = None,
    ) -> None:
        """Initialization"""
        self.price_asset: pd.DataFrame = self.check_price_df(price_asset)
        self.frequency: str = frequency
        self.min_assets: int = min_assets
        self.min_periods: int = min_periods
        self.name: str = name
        self.commission = commission
        self.currency = currency

        # account information
        self.idx: int = 0
        self.date: Any = None
        self.value: float = investment
        self.weights: pd.Series = pd.Series(dtype=float)
        self.reb_weights: pd.Series = pd.Series(dtype=float)
        self.trade_weights: pd.Series = pd.Series(dtype=float)
        self.account: Account = account or Account()

    ################################################################################################
    @property
    def value_df(self):
        """values dataframe"""
        return pd.DataFrame(
            data=self.account.value, index=self.account.date, columns=["value"]
        )

    @property
    def weights_df(self):
        """weights dataframe"""
        return pd.DataFrame(data=self.account.weights, index=self.account.date)

    @property
    def reb_weights_df(self):
        """weights dataframe"""
        return pd.DataFrame(data=self.account.reb_weights, index=self.account.date)

    @property
    def trade_weights_df(self):
        """weights dataframe"""
        return pd.DataFrame(data=self.account.trade_weights, index=self.account.date)

    ################################################################################################

    def update_book(self) -> None:
        """update the account value based on the current date"""
        prices = self.price_asset.loc[self.date]
        if not self.weights.empty:
            print(f"update book values")
            pre_prices = self.price_asset.iloc[
                self.price_asset.index.get_loc(self.date) - 1
            ]
            capitals = self.weights * self.value
            pri_return = prices / pre_prices
            new_capitals = capitals * pri_return.loc[capitals.index]
            profit_loss = new_capitals - capitals
            self.value += profit_loss.sum()
            self.weights = new_capitals / self.value

        if not self.reb_weights.empty:
            print(f"make re-allocation {self.reb_weights.to_dict()}")
            # reindex to contain the same asset.
            union_assets = self.reb_weights.index.union(self.weights.index)
            self.weights = self.weights.reindex(union_assets, fill_value=0)
            self.reb_weights = self.reb_weights.reindex(union_assets, fill_value=0)
            self.trade_weights = self.reb_weights.subtract(self.weights)
            trade_capitals = self.value * self.trade_weights
            trade_costs = trade_capitals.abs() * self.commission / 10_000
            trade_cost = trade_costs.sum()
            # update the account metrics.
            self.value -= trade_cost
            self.weights = self.reb_weights

        # do nothing if no account data.
        if self.weights.empty and self.reb_weights.empty:
            return
        # loop through all variables in account history
        for name in vars(self.account).keys():
            getattr(self.account, name).append(getattr(self, name))
        # clear the rebalancing weights.
        self.reb_weights = pd.Series(dtype=float)
        self.trade_weights = pd.Series(dtype=float)

    ################################################################################################

    def simulate(self, start: ... = None, end: ... = None) -> ...:
        """simulate historical strategy perfromance"""
        start = start or self.price_asset.index[0]
        end = end or self.price_asset.index[-1]
        reb_dates = pd.date_range(start=start, end=end, freq=self.frequency)
        for self.date in pd.date_range(start=start, end=end, freq="D"):
            print(self.date)
            if self.date in self.price_asset.index:
                self.update_book()
            if self.weights.empty or self.monitor() or self.date in reb_dates:
                if not self.reb_weights.empty:
                    continue
                self.reb_weights = self.allocate(self.date)
                print(f"rebalancing weights: {self.reb_weights.to_dict()}")
        return self

    def allocate(self, date: ... = None) -> pd.Series:
        """allocate weights based on date if date not provided use latest"""
        # pylint: disable=multiple-statements
        date = date or self.price_asset.index[-1]
        price_slice = self.price_asset.loc[:date].dropna(
            thresh=self.min_periods, axis=1
        )
        if price_slice.empty:
            return pd.Series(dtype=float)
        reb_weights = self.rebalance(price_asset=price_slice)
        if reb_weights is None:
            return pd.Series(dtype=float)
        return self.clean_weights(reb_weights, decimals=4)

    def rebalance(self, price_asset: pd.DataFrame) -> pd.Series:
        """Default rebalancing method"""
        asset = price_asset.columns
        uniform_weight = np.ones(len(asset))
        uniform_weight /= uniform_weight.sum()
        weight = pd.Series(index=asset, data=uniform_weight)
        return weight

    def monitor(self) -> bool:
        """Default monitoring method."""
        return False

    ################################################################################################
    @staticmethod
    def check_price_df(price_df: pd.DataFrame) -> pd.DataFrame:
        """Check the price_df.

        Args:
            price_df (pd.DataFrame): _description_

        Raises:
            TypeError: if price_df is not pd.DataFrame.

        Returns:
            pd.DataFrame: price_df
        """
        if not isinstance(price_df, pd.DataFrame):
            raise TypeError("price_df must be a pd.DataFrame.")
        if not isinstance(price_df.index, pd.DatetimeIndex):
            warnings.warn("converting price_df's index to pd.DatetimeIndex.")
            price_df.index = pd.to_datetime(price_df.index)
        return price_df

    @staticmethod
    def clean_weights(weights: pd.Series, decimals: int = 4) -> pd.Series:
        """Clean weights based on the number decimals and maintain the total of weights.

        Args:
            weights (pd.Series): asset weights.
            decimals (int, optional): number of decimals to be rounded for
                weight. Defaults to 4.

        Returns:
            pd.Series: clean asset weights.
        """
        # clip weight values by minimum and maximum.
        tot_weight = weights.sum().round(4)
        weights = weights.round(decimals=decimals)
        # repeat round and weight calculation.
        for _ in range(10):
            weights = weights / weights.sum() * tot_weight
            weights = weights.round(decimals=decimals)
            if weights.sum() == tot_weight:
                return weights
        # if residual remains after repeated rounding.
        # allocate the the residual weight on the max weight.
        residual = tot_weight - weights.sum()
        # !!! Error may occur when there are two max weights???
        weights[np.argmax(weights)] += np.round(residual, decimals=decimals)
        return weights


####################################################################################################
def backtest(allocation_df: pd.DataFrame, price_df: pd.DataFrame) -> BaseStrategy:
    """provide a backtested strategy.

    Args:
        allocation_df (pd.DataFrame): allocation dataframe.
        price_df (pd.DataFrame): asset price dataframe.

    Returns:
        BaseStrategy: backtested strategy.
    """

    class Backtest(BaseStrategy):
        """backtest class"""

        def rebalance(self, price_asset: pd.DataFrame) -> pd.Series:
            if self.date in allocation_df.index:
                weights = allocation_df.loc[self.date].dropna()
                return weights[weights != 0]
            return pd.Series(dtype=float)

    strategy = Backtest(price_asset=price_df, frequency="D").simulate(
        start=allocation_df.index[0]
    )

    return strategy


path = r"C:\Users\vip\OneDrive\DWS\ABL_RESULT\MLP_ALLOCATION.xlsx"

import pandas_datareader as pdr

allocation = pd.read_excel(path, "allocation", index_col=[1, 2, 0], parse_dates=True)
allocation = allocation.unstack().unstack()["weights"]

stra = "US_5"

allo = allocation[stra].dropna(how="all", axis=1)

tickers = ", ".join(allo.columns.tolist())
tickers = tickers.replace(".KS", "")
tickers = (
    tickers
    if isinstance(tickers, (list, set, tuple))
    else tickers.replace(",", " ").upper().split()
)

prices = []
if stra.startswith("KR"):
    for ticker in tickers:
        price = pdr.DataReader(ticker, "naver", start="1990-1-1").astype(float)["Close"]
        price.name = ticker + ".KS"
        prices.append(price)

    prices = pd.concat(prices, axis=1)

    prices.index = pd.to_datetime(prices.index)
if stra.startswith("US"):
    import yfinance as yf

    prices = yf.download(tickers)["Adj Close"]

result = backtest(allocation_df=allo, price_df=prices)

with pd.ExcelWriter(f"{stra}.xlsx", engine="openpyxl") as writer:
    # Save each dataframe to a separate sheet in the Excel file
    value = result.value_df
    value.index.name = "date"
    weights = result.weights_df
    weights.index.name = "date"
    al = result.reb_weights_df.dropna(how="all")
    al.index.name = "date"

    value.to_excel(writer, sheet_name="value")
    weights.to_excel(writer, sheet_name="weights")
    al.to_excel(writer, sheet_name="allocations")

value.plot()